In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns

LOADING THE DATA

In [3]:
df_full = pd.read_csv("churn_bank.csv") #if already in the repo, otherwise use the full path

Who has exited the bank?
Use `.loc` to select columns or rows meeting a condition.

Use `.describe()` to generate descriptive statistics (include those that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values).

In [4]:
df_exited = df_full.loc[df_full["Exited"]==1]
df_exited.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerId,2037.0,1.569005e+07,72692.623920,15565706.00,15627360.00,15688963.00,15753092.00,15815656.00
CreditScore,2037.0,6.453515e+02,100.321503,350.00,578.00,646.00,716.00,850.00
Age,2037.0,4.483800e+01,9.761562,18.00,38.00,45.00,51.00,84.00
Tenure,2037.0,4.932744e+00,2.936106,0.00,2.00,5.00,8.00,10.00
Balance,2037.0,9.110854e+04,58360.794816,0.00,38340.02,109349.29,131433.33,250898.09
NumOfProducts,2037.0,1.475209e+00,0.801521,1.00,1.00,1.00,2.00,4.00
HasCrCard,2037.0,6.990673e-01,0.458776,0.00,0.00,1.00,1.00,1.00
IsActiveMember,2037.0,3.608247e-01,0.480358,0.00,0.00,0.00,1.00,1.00
EstimatedSalary,2037.0,1.014657e+05,57912.418071,11.58,51907.72,102460.84,152422.91,199808.10
Exited,2037.0,1.000000e+00,0.000000,1.00,1.00,1.00,1.00,1.00


In [5]:
df_not_exited = df_full.loc[df_full["Exited"]==0]
df_not_exited.describe().T

,count,mean,std,min,25%,50%,75%,max
CustomerId,7963.0,1.569117e+07,71744.234941,15565701.00,15628818.50,15691543.00,1.575335e+07,15815690.00
CreditScore,7963.0,6.518532e+02,95.653837,405.00,585.00,653.00,7.180000e+02,850.00
Age,7963.0,3.740839e+01,10.125363,18.00,31.00,36.00,4.100000e+01,92.00
Tenure,7963.0,5.033279e+00,2.880658,0.00,3.00,5.00,7.000000e+00,10.00
Balance,7963.0,7.274530e+04,62848.040701,0.00,0.00,92072.68,1.264103e+05,221532.80
NumOfProducts,7963.0,1.544267e+00,0.509536,1.00,1.00,2.00,2.000000e+00,3.00
HasCrCard,7963.0,7.071455e-01,0.455101,0.00,0.00,1.00,1.000000e+00,1.00
IsActiveMember,7963.0,5.545649e-01,0.497045,0.00,0.00,1.00,1.000000e+00,1.00
EstimatedSalary,7963.0,9.973839e+04,57405.586966,90.07,50783.49,99645.04,1.486100e+05,199992.48
Exited,7963.0,0.000000e+00,0.000000,0.00,0.00,0.00,0.000000e+00,0.00


At first sight i don't see huge differences in the summary statistics of both groups. 

In [26]:
gender_count_E = df_exited["Gender"].value_counts()
print("Exited:")
print(gender_count_E) 

gender_count_NE = df_not_exited["Gender"].value_counts()
print("Stayed:")
print(gender_count_NE)



Exited:
Gender
Female    1139
Male       898
Name: count, dtype: int64
Stayed:
Gender
Male      4559
Female    3404
Name: count, dtype: int64


In [27]:
women_perc_E = gender_count_E.values[0]/gender_count_E.values.sum()
print(f"Percentage of women among exited people: {float(women_perc_E)}")

women_perc_NE = gender_count_NE.values[0]/gender_count_NE.values.sum()
print(f"Percentage of women among people who stayed: {float(women_perc_NE)}")

Percentage of women among exited people: 0.5591556210112911
Percentage of women among people who stayed: 0.5725229184980535


Nothing significative in the gender gap? 

(Also sarah if you see this and see that im making rookie mistakes please dont hesitate to correct me)

QUESTIONS

What is the churn rate among the bank's customers?

`nunique`: count number of distinct elements in specified axis.


In [28]:
# getting total number of customers
tot_customer = df_full["CustomerId"].nunique()
nbr_churned = df_full.loc[df_full["Exited"]==1, "CustomerId"].nunique()

churn_rate = (nbr_churned/tot_customer)*100
print(churn_rate) # around 20%

20.369999999999997


490.9180166912126
